In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
load_path = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        load_path.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-data/heart_disease_uci.csv


Loading csv to DataFrame

In [2]:
data = pd.read_csv(load_path[0])

DataFrame overwiev

In [3]:
print(data.head(), '\n', data.describe(include='all'))

   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4             normal    0   
       

We have some numerical and object values in DataFrame.

In [4]:
object_columns = data.select_dtypes(include=['object']).columns
[print(col_name, ":", data[col].unique()) for col, col_name in zip(data[object_columns], object_columns)]

sex : ['Male' 'Female']
dataset : ['Cleveland' 'Hungary' 'Switzerland' 'VA Long Beach']
cp : ['typical angina' 'asymptomatic' 'non-anginal' 'atypical angina']
fbs : [True False nan]
restecg : ['lv hypertrophy' 'normal' 'st-t abnormality' nan]
exang : [False True nan]
slope : ['downsloping' 'flat' 'upsloping' nan]
thal : ['fixed defect' 'normal' 'reversable defect' nan]


[None, None, None, None, None, None, None, None]

In [5]:
print(pd.isna(data).sum())

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64


Dropping columns ['slope', 'ca', 'thal'] because of too much NaN.

In [6]:
data = data.drop(['slope', 'ca', 'thal'], axis=1)
data = data.dropna(axis=0)
data

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,914,62,Male,VA Long Beach,asymptomatic,158.0,170.0,False,st-t abnormality,138.0,True,0.0,1
914,915,46,Male,VA Long Beach,asymptomatic,134.0,310.0,False,normal,126.0,False,0.0,2
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,1
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,2


Label encoding for categorical data.

In [7]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
# df['species']= label_encoder.fit_transform(df['species'])
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])
data


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,num
0,1,63,1,0,3,145.0,233.0,1,0,150.0,0,2.3,0
1,2,67,1,0,0,160.0,286.0,0,0,108.0,1,1.5,2
2,3,67,1,0,0,120.0,229.0,0,0,129.0,1,2.6,1
3,4,37,1,0,2,130.0,250.0,0,1,187.0,0,3.5,0
4,5,41,0,0,1,130.0,204.0,0,0,172.0,0,1.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,914,62,1,3,0,158.0,170.0,0,2,138.0,1,0.0,1
914,915,46,1,3,0,134.0,310.0,0,1,126.0,0,0.0,2
915,916,54,0,3,0,127.0,333.0,1,2,154.0,0,0.0,1
917,918,55,1,3,0,122.0,223.0,1,2,100.0,0,0.0,2


In [8]:
X = pd.DataFrame(data.iloc[:, 1:-1])
Y = pd.DataFrame(data.iloc[:, -1])

Data normalization. Normalization improved accuracy from 67% to 79%

In [9]:
X = (X - X.min()) / (X.max() - X.min()).values

In [10]:
disease_threshold = 1
Y[Y >= disease_threshold] = 1

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Learning NB model

In [12]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1.0, fit_prior=True)

In [13]:
clf.fit(X_train, np.ravel(Y_train))

MultinomialNB()

In [14]:
prediction = clf.predict(X_test)
print(prediction)

[1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1
 1 1 0 1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 1
 0 1 0 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 0
 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [15]:
accuracy = clf.score(X_test, Y_test)
print(accuracy)

0.7972972972972973


Tuning with cross validation and ROC (Receiver Operating Charachteristic).

In [16]:
from sklearn.model_selection import StratifiedKFold
k=10
k_fold = StratifiedKFold(n_splits=k)

In [17]:
smoothing_factor = list(range(1, 7))
fit_prior = [True, False]
auc_record = {}

In [18]:
from sklearn.metrics import roc_auc_score

for train_indices, test_indices in k_fold.split(X, Y):
    X_train, X_test = X.iloc[train_indices, :], X.iloc[test_indices, :]
    Y_train, Y_test = np.ravel(Y.iloc[train_indices, :]), Y.iloc[test_indices, :]
    for alpha in smoothing_factor:
        if alpha not in auc_record:
            auc_record[alpha] = {}
        for prior in fit_prior:
            clf = MultinomialNB(alpha=alpha, fit_prior=prior)
            clf.fit(X_train, Y_train)
            prediction_prob = clf.predict_proba(X_test)
            positive_prob = prediction_prob[:, 1]
            auc = roc_auc_score(Y_test, positive_prob)
            auc_record[alpha][prior] = auc + auc_record[alpha].get(prior, .0)

print(auc_record)

{1: {True: 8.449296317717371, False: 8.449296317717371}, 2: {True: 8.449293104556263, False: 8.449293104556263}, 3: {True: 8.447829509671616, False: 8.447829509671616}, 4: {True: 8.446367521367522, False: 8.446367521367522}, 5: {True: 8.44344193817878, False: 8.44344193817878}, 6: {True: 8.445634920634921, False: 8.445634920634921}}


In [19]:
print('Smoothing  A priori  Area')
for smoothing, smoothing_record in auc_record.items():
    for fit_prior, auc in smoothing_record.items():
        print(f'    {smoothing}      {fit_prior!s:<6}    {auc/k:.5f}')

Smoothing  A priori  Area
    1      True      0.84493
    1      False     0.84493
    2      True      0.84493
    2      False     0.84493
    3      True      0.84478
    3      False     0.84478
    4      True      0.84464
    4      False     0.84464
    5      True      0.84434
    5      False     0.84434
    6      True      0.84456
    6      False     0.84456


We have 4 models with the same area under ROC curve.

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train, np.ravel(Y_train))
pos_prob = clf.predict_proba(X_test)[:,1]
print("Area under ROC in best model is: ", roc_auc_score(Y_test, pos_prob))
accuracy = clf.score(X_test, Y_test)
print(accuracy)

Area under ROC in best model is:  0.8409132420091324
0.7972972972972973
